# <center> *Big Data project* </center> 
### <center> *Black Hair model* </center>
_<center>M2 Statistics & Econometrics, 2019-2020</center>_


In [0]:
import os
import zipfile
import random
import pickle
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import expand_dims
from PIL import Image
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
!pip uninstall tensorflow -gpu -y
!pip install tensorflow-gpu==2.1.0
import tensorflow as tf
print(tf.__version__)
import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import AveragePooling2D, MaxPool2D, Conv2D, MaxPooling2D, InputLayer 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LambdaCallback
from keras.utils import np_utils, to_categorical
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.models import load_model


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -g
2.1.0


Using TensorFlow backend.


In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with zipfile.ZipFile('/content/drive/My Drive/Big Data/Projet/celeba-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/img/')

In [0]:
path = '/content/img/img_align_celeba/img_align_celeba/'
samp = pickle.load(open('/content/drive/My Drive/Big Data/Projet/Listes sample/Black_Hair.pkl', 'rb'))

In [0]:
target = []
data = np.zeros((len(samp), 32, 32, 3)) #dimension of the images
count = 0
for file in os.listdir(path) :
  if file in samp :
    image = Image.open(path + file)
    image = image.resize((32, 32))
    data[count] = np.array(image)
    target.append(file)
    if count%1000 == 0 :
      print(count)
    count += 1
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [0]:
df_attr = pd.read_csv('/content/img/list_attr_celeba.csv')
df_attr.replace(to_replace = -1, value = 0, inplace = True)
black_hair = df_attr[['image_id', 'Black_Hair']].set_index('image_id')
black_hair = black_hair.loc[target]
print(black_hair.shape)
black_hair.head()

(20000, 1)


,Black_Hair
image_id,
027984.jpg,1
174024.jpg,0
194156.jpg,1
198131.jpg,1
012318.jpg,0


### **Baseline model**

In [0]:
target = to_categorical(black_hair.Black_Hair)
print(target.shape)
print(data.shape)

(20000, 2)
(20000, 32, 32, 3)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                    test_size = 0.2, random_state = 0)
print("Train set dimension: ", X_train.shape, y_train.shape)
print("Test set dimension: ", X_test.shape, y_test.shape)

Train set dimension:  (16000, 32, 32, 3) (16000, 2)
Test set dimension:  (4000, 32, 32, 3) (4000, 2)


In [0]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape = (32, 32, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

model.compile(optimizers.Adam(lr = 0.001), loss = "categorical_crossentropy",
              metrics = ["accuracy"])

In [0]:
model.fit(X_train, y_train, batch_size = 32, epochs = 10, 
          validation_data = (X_test, y_test))

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 28s 2ms/sample - loss: 1.9665 - accuracy: 0.7383 - val_loss: 0.4714 - val_accuracy: 0.7860
Epoch 2/10
16000/16000 [==============================] - 27s 2ms/sample - loss: 0.4583 - accuracy: 0.7928 - val_loss: 0.4149 - val_accuracy: 0.8177
Epoch 3/10
16000/16000 [==============================] - 28s 2ms/sample - loss: 0.4158 - accuracy: 0.8157 - val_loss: 0.3952 - val_accuracy: 0.8205
Epoch 4/10
16000/16000 [==============================] - 28s 2ms/sample - loss: 0.4058 - accuracy: 0.8192 - val_loss: 0.3847 - val_accuracy: 0.8288
Epoch 5/10
16000/16000 [==============================] - 28s 2ms/sample - loss: 0.3865 - accuracy: 0.8326 - val_loss: 0.4028 - val_accuracy: 0.8317
Epoch 6/10
16000/16000 [==============================] - 28s 2ms/sample - loss: 0.3813 - accuracy: 0.8335 - val_loss: 0.4050 - val_accuracy: 0.8192
Epoch 7/10
16000/16000 [==============================] -

### **Model**

In [0]:
target = black_hair.Black_Hair
print(target.shape)
print(data.shape)

(20000,)
(20000, 32, 32, 3)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2,
                                                    random_state = 0)
print("Train set dimension: ", X_train.shape, y_train.shape)
print("Test set dimension: ", X_test.shape, y_test.shape)

Train set dimension:  (16000, 32, 32, 3) (16000,)
Test set dimension:  (4000, 32, 32, 3) (4000,)


In [0]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizers.Adam(lr = 0.001), loss = "binary_crossentropy", 
              metrics = ["accuracy"])

In [0]:
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.2, patience = 1, 
                              min_lr = 0.001)

save_models = "/content/drive/My Drive/Big Data/Projet/Models/"
model_name = "Best_Black_Hair_model.hdf5"
file_path = save_models + model_name
checkpoint = ModelCheckpoint(file_path, monitor = 'val_accuracy', verbose = 1, 
                             save_best_only = True, mode = 'max')

callbacks = [checkpoint, reduce_lr]

In [0]:
model.fit(X_train, y_train, batch_size = 32, epochs = 10, 
          validation_data = (X_test, y_test), callbacks = callbacks)

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
15968/16000 [============================>.] - ETA: 0s - loss: 0.6645 - accuracy: 0.7452
Epoch 00001: val_accuracy improved from -inf to 0.78175, saving model to /content/drive/My Drive/Big Data/Projet/Models/Best_Black_Hair_model.hdf5
16000/16000 [==============================] - 59s 4ms/sample - loss: 0.6642 - accuracy: 0.7453 - val_loss: 0.4479 - val_accuracy: 0.7818
Epoch 2/10
15968/16000 [============================>.] - ETA: 0s - loss: 0.4213 - accuracy: 0.8156
Epoch 00002: val_accuracy improved from 0.78175 to 0.79625, saving model to /content/drive/My Drive/Big Data/Projet/Models/Best_Black_Hair_model.hdf5
16000/16000 [==============================] - 58s 4ms/sample - loss: 0.4213 - accuracy: 0.8158 - val_loss: 0.4169 - val_accuracy: 0.7962
Epoch 3/10
15968/16000 [============================>.] - ETA: 0s - loss: 0.3879 - accuracy: 0.8327
Epoch 00003: val_accuracy improved from 0.79625 to 0.83200, saving model to /c